In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

%pylab inline
import seaborn as sns

path='./data/data137263/'

Populating the interactive namespace from numpy and matplotlib


In [2]:
train=pd.read_csv(path+'pubg_train.csv.zip')
test=pd.read_csv(path+'pubg_test.csv.zip')

train['type']='trian'
test['type']='test'
data=pd.concat([train,test])

In [3]:
tmpcols=['kill_distance_x_min','kill_distance_x_max', 'kill_distance_y_min', 'kill_distance_y_max']
data[tmpcols]=data[tmpcols]+1
data[tmpcols]=data[tmpcols].fillna(0)

In [4]:
tmp=data.groupby(['match_id','team_id'])['player_dist_ride'].min().reset_index()

In [5]:
process_tmp=data[['match_id','team_id']]
process_cols=['player_assists','player_dbno','player_dist_ride','player_dist_walk',
'player_dmg','player_kills','kill_distance_x_min','kill_distance_x_max','kill_distance_y_min','kill_distance_y_max']

tmp=data.groupby(['match_id','team_id'])[process_cols].min().reset_index()
# tmp.columns = ['match_id','team_id']+[i+'_min' for i in process_cols]
process_tmp=process_tmp.merge(tmp,on=['match_id','team_id'],how='left')

tmp=data.groupby(['match_id','team_id'])[process_cols].max().reset_index()
# tmp.columns = ['match_id','team_id']+[i+'_max' for i in process_cols]
process_tmp=process_tmp.merge(tmp,on=['match_id','team_id'],how='left')

tmp=data.groupby(['match_id','team_id'])[process_cols].sum().reset_index()
# tmp.columns = ['match_id','team_id']+[i+'_sum' for i in process_cols]
process_tmp=process_tmp.merge(tmp,on=['match_id','team_id'],how='left')

# tmp=data.groupby(['match_id','team_id'])[process_cols].mean().reset_index()
# tmp.columns = ['match_id','team_id']+[i+'_mean' for i in process_cols]
# process_tmp=process_tmp.merge(tmp,on=['match_id','team_id'],how='left')

process_tmp=process_tmp.drop_duplicates()
data=data[['match_id','team_id','game_size','party_size','team_placement','type']]
data=data.drop_duplicates()
data=data.merge(process_tmp,on=['match_id','team_id'],how='left')

In [6]:
train=data[data['type']=='trian']
test=data[data['type']=='test']

# val=train.tail(500000)
# val['type']='val'
# train=train.head(1000000)

In [7]:
train['count']=train.groupby('match_id')['team_placement'].transform('count')
test['count']=test.groupby('match_id')['team_placement'].transform('count')
# val['count']=val.groupby('match_id')['team_placement'].transform('count')

tmp=train[['match_id','count']]
tmp=tmp.drop_duplicates(keep='first')
g_train=tmp['count'].values.tolist()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
lgb_cols=['match_id', 'team_id', 'game_size', 'party_size', 
       'player_assists_x', 'player_dbno_x', 'player_dist_ride_x',
       'player_dist_walk_x', 'player_dmg_x', 'player_kills_x',
       'kill_distance_x_min_x', 'kill_distance_x_max_x',
       'kill_distance_y_min_x', 'kill_distance_y_max_x', 'player_assists_y',
       'player_dbno_y', 'player_dist_ride_y', 'player_dist_walk_y',
       'player_dmg_y', 'player_kills_y', 'kill_distance_x_min_y',
       'kill_distance_x_max_y', 'kill_distance_y_min_y',
       'kill_distance_y_max_y', 'player_assists', 'player_dbno',
       'player_dist_ride', 'player_dist_walk', 'player_dmg', 'player_kills',
       'kill_distance_x_min', 'kill_distance_x_max', 'kill_distance_y_min',
       'kill_distance_y_max']

# lgb_cols=['match_id', 'team_id', 'game_size', 'party_size',
#        'player_assists_min', 'player_dbno_min', 'player_dist_ride_min',
#        'player_dist_walk_min', 'player_dmg_min', 'player_kills_min',
#        'kill_distance_x_min_min', 'kill_distance_x_max_min',
#        'kill_distance_y_min_min', 'kill_distance_y_max_min',
#        'player_assists_max', 'player_dbno_max', 'player_dist_ride_max',
#        'player_dist_walk_max', 'player_dmg_max', 'player_kills_max',
#        'kill_distance_x_min_max', 'kill_distance_x_max_max',
#        'kill_distance_y_min_max', 'kill_distance_y_max_max',
#        'player_assists_sum', 'player_dbno_sum', 'player_dist_ride_sum',
#        'player_dist_walk_sum', 'player_dmg_sum', 'player_kills_sum',
#        'kill_distance_x_min_sum', 'kill_distance_x_max_sum',
#        'kill_distance_y_min_sum', 'kill_distance_y_max_sum',
#        'player_assists_mean', 'player_dbno_mean', 'player_dist_ride_mean',
#        'player_dist_walk_mean', 'player_dmg_mean', 'player_kills_mean',
#        'kill_distance_x_min_mean', 'kill_distance_x_max_mean',
#        'kill_distance_y_min_mean', 'kill_distance_y_max_mean', 'count']

# lgb_cols=['match_id', 'team_id', 'game_size', 'party_size', 
#        'player_assists_min', 'player_dbno_min', 'player_dist_ride_min',
#        'player_dist_walk_min', 'player_dmg_min', 'player_kills_min',
#        'kill_distance_x_min_min', 'kill_distance_x_max_min',
#        'kill_distance_y_min_min', 'kill_distance_y_max_min',
#        'player_assists_max', 'player_dbno_max', 'player_dist_ride_max',
#        'player_dist_walk_max', 'player_dmg_max', 'player_kills_max',
#        'kill_distance_x_min_max', 'kill_distance_x_max_max',
#        'kill_distance_y_min_max', 'kill_distance_y_max_max',
#        'player_assists_sum', 'player_dbno_sum', 'player_dist_ride_sum',
#        'player_dist_walk_sum', 'player_dmg_sum', 'player_kills_sum',
#        'kill_distance_x_min_sum', 'kill_distance_x_max_sum',
#        'kill_distance_y_min_sum', 'kill_distance_y_max_sum', 'count']

In [9]:
# 模型及参数的定义
lgb_Ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,objective='regression',
                            max_depth=-1, n_estimators=3000, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2022, n_jobs= 16)

In [10]:
lgb_Ranker.fit(train[lgb_cols], train['team_placement'],group=g_train)

LGBMRanker(colsample_bytree=0.7, learning_rate=0.01, min_child_weight=50,
           n_estimators=3000, n_jobs=16, objective='regression',
           random_state=2022, reg_lambda=1, subsample=0.7, subsample_freq=1)

In [11]:
# 模型预测
test['rate'] = lgb_Ranker.predict(test[lgb_cols])
ygq=test[['match_id','team_id','rate']]

test=pd.read_csv(path+'pubg_test.csv.zip',usecols=['match_id','team_id'])
test=pd.merge(test,ygq,on=['match_id','team_id'],how='left')

ygq=test['rate'].tolist()
pd.DataFrame({
    'team_placement': ygq
}).to_csv('submission.csv', index=None)

!zip submission.zip submission.csv

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


updating: submission.csv (deflated 72%)


In [12]:
# from sklearn.metrics import mean_absolute_error

# # 模型验证
# val['rate'] = lgb_Ranker.predict(val[lgb_cols])
# ygq=val[['match_id','team_id','rate']]

# val=pd.read_csv(path+'pubg_train.csv.zip',usecols=['match_id','team_id','team_placement'])
# val=val.tail(500000)
# ygq_true=val['team_placement'].tolist()
# val=val[['match_id','team_id']]
# val=pd.merge(val,ygq,on=['match_id','team_id'],how='left')
# ygq=val['rate'].tolist()

# print(100 - mean_absolute_error(ygq_true, ygq))

In [13]:
# # 五折交叉验证，这里的五折交叉是以用户为目标进行五折划分
# #  这一部分与前面的单独训练和验证是分开的
# def get_kfold_users(trn_df, n=5):
#     user_ids = trn_df['match_id'].unique()
#     user_set = [user_ids[i::n] for i in range(n)]
#     return user_set

# k_fold = 5
# user_set = get_kfold_users(train, n=k_fold)

# score_list = []
# score_df = trn_df[['match_id', 'team_id','team_placement']]
# sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

# # 五折交叉验证，并将中间结果保存用于staking
# for n_fold, valid_user in enumerate(user_set):
#     train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user
#     valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]
    
#     # 训练集与验证集的用户分组
#     train_idx.sort_values(by=['user_id'], inplace=True)
#     g_train = train_idx.groupby(['user_id'], as_index=False).count()["label"].values
    
#     valid_idx.sort_values(by=['user_id'], inplace=True)
#     g_val = valid_idx.groupby(['user_id'], as_index=False).count()["label"].values
    
#     # 定义模型
#     lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
#                             max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
#                             learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16)  
#     # 训练模型
#     lgb_ranker.fit(train_idx[lgb_cols], train_idx['label'], group=g_train,
#                    eval_set=[(valid_idx[lgb_cols], valid_idx['label'])], eval_group= [g_val], 
#                    eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, )
    
#     # 预测验证集结果
#     valid_idx['pred_score'] = lgb_ranker.predict(valid_idx[lgb_cols], num_iteration=lgb_ranker.best_iteration_)
    
#     # 对输出结果进行归一化
#     valid_idx['pred_score'] = valid_idx[['pred_score']].transform(lambda x: norm_sim(x))
    
#     valid_idx.sort_values(by=['user_id', 'pred_score'])
#     valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
#     # 将验证集的预测结果放到一个列表中，后面进行拼接
#     score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])
    
#     # 如果是线上测试，需要计算每次交叉验证的结果相加，最后求平均
#     if not offline:
#         sub_preds += lgb_ranker.predict(tst_user_item_feats_df_rank_model[lgb_cols], lgb_ranker.best_iteration_)
    

    
    
    
# # 保存训练集交叉验证产生的新特征
# score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_ranker_feats.csv', index=False)
    
# # 测试集的预测结果，多次交叉验证求平均,将预测的score和对应的rank特征保存，可以用于后面的staking，这里还可以构造其他更多的特征
# tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
# tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
# tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
# tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

# # 保存测试集交叉验证的新特征
# tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_ranker_feats.csv', index=False)
